In [42]:
import argparse
import sys
sys.path.append('../src')
import loader
import rnn_encoder_decoder
from torchtext import data
import torch
from collections import Counter
#import matplotlib.pyplot as plt
#%matplotlib inline
import torch.nn as nn
import numpy as np

In [4]:
from elmoformanylangs import Embedder

In [9]:
torch.cuda.set_device(1)

In [10]:
chinese_embedder = Embedder('../../ELMoForManyLangs/179')

In [13]:
# FileNotFoundError: [Errno 2] No such file or directory: 
# '/Users/yijialiu/work/projects/conll2018/models/word_elmo/cnn_50_100_512_4096_sample.json'
vietnamese_embedder = Embedder('../../ELMoForManyLangs/178')
# FileNotFoundError: [Errno 2] No such file or directory: 
# '/Users/yijialiu/work/projects/conll2018/models/word_elmo/cnn_50_100_512_4096_sample.json'
english_embedder = Embedder('../../ELMoForManyLangs/144')

In [14]:
simplified_chinese_embedder = Embedder('../../ELMoForManyLangs/zhs.model')

In [15]:
# MUST USE GPU (?)
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = False

In [16]:
train_data, val_data, test_data, src, trg = loader.load_data(args)

Most common source vocabs: [('的', 272264), ('我', 83174), ('是', 71290), ('我们', 68696), ('在', 55727), ('了', 52197), ('你', 40824), ('这', 36049), ('一个', 33590), ('他们', 25256)]
Source vocab size: 47127
Most common english vocabs: [('the', 178698), ('and', 128020), ('to', 106119), ('of', 98919), ('a', 90534), ('that', 80454), ('i', 68877), ('in', 66951), ('it', 64198), ('you', 61161)]
English vocab size: 28791


In [57]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = True
c_train_data, c_val_data, c_test_data, c_src, c_trg = loader.load_data(args)

Most common source vocabs: [('的', 281920), ('我', 154809), ('一', 124419), ('是', 124104), ('们', 120231), ('这', 97418), ('个', 84887), ('在', 80827), ('有', 73956), ('人', 63944)]
Source vocab size: 4168
Most common english vocabs: [('the', 178698), ('and', 128020), ('to', 106119), ('of', 98919), ('a', 90534), ('that', 80454), ('i', 68877), ('in', 66951), ('it', 64198), ('you', 61161)]
English vocab size: 28791


In [74]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.source_lang = 'vi'
args.data = '../data/iwslt-vi-en/'
v_train_data, v_val_data, v_test_data, v_src, v_trg = loader.load_data(args)

Most common source vocabs: [('là', 51451), ('và', 47993), ('một', 40378), ('tôi', 38381), ('những', 37809), ('của', 36330), ('có', 26166), ('bạn', 26111), ('đó', 25229), ('không', 25169)]
Source vocab size: 20099
Most common english vocabs: [('the', 109723), ('and', 79673), ('to', 65979), ('of', 60510), ('a', 55374), ('that', 49320), ('i', 43629), ('in', 41318), ('it', 39058), ('you', 37438)]
English vocab size: 22429


In [55]:
def save_embedding(lang, given_embedder, output_name):
    batched_itos = []

    for i in range(len(lang.vocab.itos)):
        if i % 32 == 0:
            if i != 0:
                batched_itos.append(current_batch)
            current_batch = []
        current_batch.append(lang.vocab.itos[i])
    batched_itos.append(current_batch)
    
    # change the vectors
    batched_itos[0][0], batched_itos[0][1], batched_itos[0][2], batched_itos[0][3] = '<oov>', '<pad>', '<bos>', '<eos>'
    
    torch.cuda.set_device(1)
    tokenized_itos = given_embedder.sents2elmo(batched_itos, output_layer=0)
    
    embedding = nn.Embedding(
            len(lang.vocab.itos), 
            1024, 
            padding_idx = 1
        )
    concatenated = np.concatenate(tokenized_itos)
    for i, each in enumerate(concatenated):
        if np.array_equal(each, concatenated[0]):
            print(i)
    
    embedding.load_state_dict({'weight': torch.FloatTensor(concatenated)})
    torch.save(embedding.state_dict(), output_name)
    


In [56]:
save_embedding(src, chinese_embedder, 'zh_en_chinese_words_traditional.pt')
save_embedding(src, simplified_chinese_embedder, 'zh_en_chinese_words_simplified.pt')
save_embedding(trg, english_embedder, 'zh_en_english_words.pt')

0
0
0


In [58]:
save_embedding(c_src, chinese_embedder, 'zh_en_chinese_chars_traditional.pt')
save_embedding(c_src, simplified_chinese_embedder, 'zh_en_chinese_chars_simplified.pt')

0
0


In [75]:
save_embedding(v_src, vietnamese_embedder, 'vi_en_vietnamese_words.pt')
save_embedding(v_trg, english_embedder, 'vi_en_english_words.pt')

0
0


In [76]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.source_lang = 'vi'
args.data = '../data/short-sentences-vi-en/'
sv_train_data, sv_val_data, sv_test_data, sv_src, sv_trg = loader.load_data(args)

Most common source vocabs: [('là', 8210), ('tôi', 3628), ('không', 3530), ('Và', 3413), ('một', 3290), ('có', 2911), ('đó', 2883), ('Tôi', 2827), ('của', 2605), ('này', 2532)]
Source vocab size: 6495
Most common english vocabs: [('the', 10506), ('it', 7023), ('and', 6401), ('&apos;s', 6272), ('a', 5965), ('to', 5712), ('i', 5688), ('is', 5612), ('you', 5432), ('of', 4902)]
English vocab size: 5897


In [77]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = True
args.data = '../data/short-sentences-zh-en/'
sc_train_data, sc_val_data, sc_test_data, sc_src, sc_trg = loader.load_data(args)

Most common source vocabs: [('是', 4844), ('我', 4737), ('的', 4194), ('这', 4076), ('谢', 3401), ('们', 2736), ('了', 2164), ('不', 1997), ('一', 1866), ('有', 1852)]
Source vocab size: 2042
Most common english vocabs: [('you', 3654), ('it', 3517), ('the', 3275), ('&apos;s', 3208), ('i', 2877), ('and', 2576), ('is', 2350), ('that', 2311), ('a', 2121), ('to', 1972)]
English vocab size: 2592


In [78]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = False
args.data = '../data/short-sentences-zh-en/'
s_train_data, s_val_data, s_test_data, s_src, s_trg = loader.load_data(args)

Most common source vocabs: [('的', 3925), ('我', 3262), ('是', 3127), ('这', 2377), ('了', 1938), ('我们', 1417), ('谢谢', 1356), ('你', 1340), ('它', 756), ('什么', 754)]
Source vocab size: 3208
Most common english vocabs: [('you', 3654), ('it', 3517), ('the', 3275), ('&apos;s', 3208), ('i', 2877), ('and', 2576), ('is', 2350), ('that', 2311), ('a', 2121), ('to', 1972)]
English vocab size: 2592


In [79]:
save_embedding(s_src, chinese_embedder, 'short_sentences_zh_en_chinese_words_traditional.pt')
save_embedding(s_src, simplified_chinese_embedder, 'short_sentences_zh_en_chinese_words_simplified.pt')
save_embedding(s_trg, english_embedder, 'short_sentences_zh_en_english_words.pt')
save_embedding(sc_src, chinese_embedder, 'short_sentences_zh_en_chinese_chars_traditional.pt')
save_embedding(sc_src, simplified_chinese_embedder, 'short_sentences_zh_en_chinese_chars_simplified.pt')
save_embedding(sv_src, vietnamese_embedder, 'short_sentences_vi_en_vietnamese_words.pt')
save_embedding(sv_trg, english_embedder, 'short_sentences_vi_en_english_words.pt')

0
0
0
0
0
0
0


In [80]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.source_lang = 'vi'
args.data = '../data/short-sentences-vi-en/'
smallv_train_data, smallv_val_data, smallv_test_data, smallv_src, smallv_trg = loader.load_data(args)

Most common source vocabs: [('là', 8210), ('tôi', 3628), ('không', 3530), ('Và', 3413), ('một', 3290), ('có', 2911), ('đó', 2883), ('Tôi', 2827), ('của', 2605), ('này', 2532)]
Source vocab size: 6495
Most common english vocabs: [('the', 10506), ('it', 7023), ('and', 6401), ('&apos;s', 6272), ('a', 5965), ('to', 5712), ('i', 5688), ('is', 5612), ('you', 5432), ('of', 4902)]
English vocab size: 5897


In [81]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = True
args.data = '../data/short-sentences-zh-en/'
smallc_train_data, smallc_val_data, smallc_test_data, smallc_src, smallc_trg = loader.load_data(args)

Most common source vocabs: [('是', 4844), ('我', 4737), ('的', 4194), ('这', 4076), ('谢', 3401), ('们', 2736), ('了', 2164), ('不', 1997), ('一', 1866), ('有', 1852)]
Source vocab size: 2042
Most common english vocabs: [('you', 3654), ('it', 3517), ('the', 3275), ('&apos;s', 3208), ('i', 2877), ('and', 2576), ('is', 2350), ('that', 2311), ('a', 2121), ('to', 1972)]
English vocab size: 2592


In [82]:
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
args.cpu = True
args.split_chinese_into_characters = False
args.data = '../data/short-sentences-zh-en/'
small_train_data, small_val_data, small_test_data, small_src, small_trg = loader.load_data(args)

Most common source vocabs: [('的', 3925), ('我', 3262), ('是', 3127), ('这', 2377), ('了', 1938), ('我们', 1417), ('谢谢', 1356), ('你', 1340), ('它', 756), ('什么', 754)]
Source vocab size: 3208
Most common english vocabs: [('you', 3654), ('it', 3517), ('the', 3275), ('&apos;s', 3208), ('i', 2877), ('and', 2576), ('is', 2350), ('that', 2311), ('a', 2121), ('to', 1972)]
English vocab size: 2592


In [83]:
save_embedding(s_src, chinese_embedder, 'small_zh_en_chinese_words_traditional.pt')
save_embedding(s_src, simplified_chinese_embedder, 'small_zh_en_chinese_words_simplified.pt')
save_embedding(s_trg, english_embedder, 'small_zh_en_english_words.pt')
save_embedding(sc_src, chinese_embedder, 'small_zh_en_chinese_chars_traditional.pt')
save_embedding(sc_src, simplified_chinese_embedder, 'small_zh_en_chinese_chars_simplified.pt')
save_embedding(sv_src, vietnamese_embedder, 'small_vi_en_vietnamese_words.pt')
save_embedding(sv_trg, english_embedder, 'small_vi_en_english_words.pt')

0
0
0
0
0
0
0


In [84]:
for each in trg.vocab.itos:
    if '&' in each:
        print(each)

&apos;s
&quot;
&apos;t
&apos;re
&apos;ve
&apos;m
&apos;ll
&apos;d
&apos;
&#91;
&#93;
&amp;
&apos;clock
&apos;all
&apos;s-eye
&apos;am
&apos;n
&apos;i
&apos;ivoire
&apos;mon
&apos;neill
&apos;an
&apos;oeil
&apos;a
&apos;andrea
&apos;est
&apos;toole
&apos;ts
&apos;walking
&apos;état


In [85]:
english_embedder

In [20]:
batched_itos = []

for i in range(len(src.vocab.itos)):
    if i % 32 == 0:
        if i != 0:
            batched_itos.append(current_batch)
        current_batch = []
    current_batch.append(src.vocab.itos[i])
batched_itos.append(current_batch)
#for each in src.vocab.itos:

In [33]:
batched_itos[0][0], batched_itos[0][1], batched_itos[0][2], batched_itos[0][3]

('<unk>', '<pad>', '<sos>', '<eos>')

In [69]:
batched_itos[0][0], batched_itos[0][1], batched_itos[0][2], batched_itos[0][3] = '<oov>', '<pad>', '<bos>', '<eos>'

In [21]:
len(batched_itos), 32*len(batched_itos), len(src.vocab.itos)

(1473, 47136, 47127)

In [36]:
# WRONG! unk eos should now be oov? and bos
torch.cuda.set_device(1)
tokenized_itos_ver2 = chinese_embedder.sents2elmo(batched_itos, output_layer=0)

In [37]:
len(tokenized_itos_ver2), tokenized_itos_ver2[0].shape, tokenized_itos_ver2[-1].shape

(1473, (32, 1024), (23, 1024))

In [39]:
embedding = nn.Embedding(
            47127, 
            1024, 
            padding_idx = 1
        )

In [41]:
type(tokenized_itos_ver2[0])

numpy.ndarray

In [43]:
concatenated = np.concatenate(tokenized_itos_ver2)

In [64]:
concatenated.shape

(47127, 1024)

In [45]:
embedding.load_state_dict({'weight': torch.FloatTensor(concatenated)})

In [46]:
torch.save(embedding.state_dict(), 'chinese_words_traditional.pt')

In [53]:
for i, each in enumerate(concatenated):
    if np.array_equal(each, concatenated[0]):
        print(i)

0


In [59]:
batched_itos = []

for i in range(len(src.vocab.itos)):
    if i % 16 == 0:
        if i != 0:
            batched_itos.append(current_batch)
        current_batch = []
    current_batch.append(src.vocab.itos[i])
batched_itos.append(current_batch)
#for each in src.vocab.itos:

In [70]:
# WRONG! unk eos should now be oov? and bos
torch.cuda.set_device(1)
tokenized_itos_ver2 = chinese_embedder.sents2elmo(batched_itos, output_layer=0)

In [71]:
tokenized_itos_ver2[0][0]

array([ 0.13876027, -0.19889045, -0.03158821, ..., -0.5328649 ,
        0.05743121, -0.40173307], dtype=float32)

In [72]:
concatenated[0]

array([ 0.13876027, -0.19889039, -0.03158827, ..., -0.53286487,
        0.05743124, -0.4017331 ], dtype=float32)

In [65]:
concatenated[1], tokenized_itos_ver2[0][1]

(array([ 0.01102709, -0.35866517,  0.10903755, ..., -0.5248197 ,
        -0.10231876, -0.4093064 ], dtype=float32),
 array([ 0.01102707, -0.35866526,  0.10903756, ..., -0.52481973,
        -0.10231876, -0.40930653], dtype=float32))

In [73]:
concatenated[2], tokenized_itos_ver2[0][2]

(array([  0.05259107, -12.8854065 ,   1.1188823 , ...,  -5.976108  ,
          2.2683213 ,  -4.609073  ], dtype=float32),
 array([  0.05259109, -12.8854065 ,   1.1188822 , ...,  -5.9761086 ,
          2.268321  ,  -4.6090736 ], dtype=float32))

In [67]:
concatenated[3], tokenized_itos_ver2[0][3]

(array([-0.5495319 , -2.7702649 , -0.20382443, ..., -1.190669  ,
        -0.04754354, -0.608676  ], dtype=float32),
 array([-0.5495319 , -2.770265  , -0.20382443, ..., -1.190669  ,
        -0.04754355, -0.60867596], dtype=float32))

In [68]:
concatenated[4], tokenized_itos_ver2[0][4]

(array([-2.0089209e-02, -7.5419992e-01,  5.4156780e-04, ...,
         2.0233115e-02,  5.6951560e-02,  1.6582429e-01], dtype=float32),
 array([-2.0089209e-02, -7.5419986e-01,  5.4156780e-04, ...,
         2.0233115e-02,  5.6951560e-02,  1.6582429e-01], dtype=float32))

In [87]:
english_embedder.model.token_embedder.word_emb_layer.word2id

{'<oov>': 0,
 '<bos>': 1,
 '<eos>': 2,
 '<pad>': 3,
 ',': 4,
 'the': 5,
 '.': 6,
 'of': 7,
 'and': 8,
 'to': 9,
 'a': 10,
 'in': 11,
 '-': 12,
 ')': 13,
 '(': 14,
 ':': 15,
 'is': 16,
 'for': 17,
 '"': 18,
 'on': 19,
 'I': 20,
 'The': 21,
 'that': 22,
 'with': 23,
 'was': 24,
 "'s": 25,
 'by': 26,
 'as': 27,
 'at': 28,
 'it': 29,
 'you': 30,
 'from': 31,
 'are': 32,
 'be': 33,
 'this': 34,
 'or': 35,
 'have': 36,
 'not': 37,
 'an': 38,
 ';': 39,
 'his': 40,
 'he': 41,
 'but': 42,
 '...': 43,
 '?': 44,
 'has': 45,
 'your': 46,
 '/': 47,
 'will': 48,
 '|': 49,
 'all': 50,
 'can': 51,
 '!': 52,
 'one': 53,
 'In': 54,
 '1': 55,
 'they': 56,
 "'": 57,
 'which': 58,
 '&': 59,
 'more': 60,
 'do': 61,
 'were': 62,
 "n't": 63,
 'my': 64,
 '’s': 65,
 'their': 66,
 'about': 67,
 'It': 68,
 'also': 69,
 'up': 70,
 'A': 71,
 'we': 72,
 'had': 73,
 'out': 74,
 '2': 75,
 'who': 76,
 '$': 77,
 'would': 78,
 'time': 79,
 'so': 80,
 'been': 81,
 'This': 82,
 'like': 83,
 '–': 84,
 'first': 85,
 'if': 86

In [93]:
for key in english_embedder.model.token_embedder.word_emb_layer.word2id:
    if key.startswith('&'):
        print(key)

&
&gt
&quot
&#
&lt
&nbsp
&gt;&gt
&T
&amp
&B
&#39
&gt;&gt;&gt
&apos
&&
&amp;&amp
&P
&S
&N
&gt;&gt;&gt;&gt
&gt;&gt;&gt;&gt;&gt
&R
&O
&W
&G
&c.
&c
&&(
&L
&C
&M
&D
&H
&&gu_ad_ph_geoc==ph_geoc)
&&gu_ad_ph_geoc_real
&mdash
&nbsp;&nbsp
&#x000A
&ndash
&A
&F
&lt;-&gt
&&_gaq.push
&raquo
&#40
&#39;s
&gt;&gt;&gt;&gt;&gt;&gt
&#039
&#39;method&#39
&V
&lt;address@hidden&gt
&#34
&bull
&#39;
&nbsp;&nbsp;&nbsp
&Q
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;


In [94]:
for key in english_embedder.model.token_embedder.word_emb_layer.word2id:
    if 'quot' in key:
        print(key)

quote
quotes
quot
&quot
quoted
quotation
quoting
.&quot
quotations
quota
quotas
,&quot
quotient
blockquote
(&quot
?&quot
;&quot
replyquote
Pequot
!&quot
yes&quot
href=&quot
Clicquot
quote:Anonymous
unquoted
Wikiquote
_ENV[&quot
Clayoquot
\&quot
class=&quot;full-spec-link&quot
device.quote
aliquot
...&quot
quotidian
misquoted
TEST_PHP_SRCDIR&quot
\n&quot
[&quot
C&quot
quotients
IS&quot
A2=&quot
mode&quot
